In [1]:
from Laba3 import Table, KronMethod

In [57]:
import numpy as np
import pandas as pd
import copy

class KronMethod:
    def __init__(self, table: Table, task_vector, type='random', CMP=False):
        if CMP == False:
            self.table = self.initial_random_distribution(table, task_vector, type)
            #print(self.table._matrix)
        else:
            self.table = self.find_critical_way(table, task_vector, table.shape[0])
            #print(self.table._matrix)
        if type == 'min':
            print("До сортировки по убыванию: ")
            self.table.get_table()
            row_sums = np.sum(self.table._matrix, axis=1)
            sorted_indices = np.argsort(row_sums)[::-1]
            sorted_matrix = self.table._matrix[sorted_indices]
            self.table._matrix = sorted_matrix[::-1]
        self._result = np.zeros(table.shape[0])
        
    
    def calculate(self):
        table = copy.copy(self.table)
        while True:
            sum_vector = self.get_sum_worktime_of_processes(table)
            min_sum_pos = np.argmin(sum_vector)
            max_sum_pos = np.argmax(sum_vector)

            delta = sum_vector[max_sum_pos] - sum_vector[min_sum_pos]
            print("Дельта: ", delta)
            is_found = False
            lower_then_delta = self.__get_lower_then_delta(table._matrix[max_sum_pos], delta)
            if lower_then_delta is not None:
                index_to_zero = lower_then_delta[0]
                table._matrix[max_sum_pos][index_to_zero] = 0
                min_argmin = np.argmin(table._matrix[min_sum_pos])
                table._matrix[min_sum_pos][min_argmin] = lower_then_delta[1]
                continue
            else:
                for i in range(len(table._matrix[max_sum_pos])):
                    for j in range(len(table._matrix[min_sum_pos])):
                        if is_found:
                            break

                        if table._matrix[max_sum_pos][i] <= table._matrix[min_sum_pos][j]:
                            continue 

                        if table._matrix[max_sum_pos][i] - table._matrix[min_sum_pos][j] < delta:
                            table._matrix[max_sum_pos][i], table._matrix[min_sum_pos][j] = table._matrix[min_sum_pos][j], table._matrix[max_sum_pos][i]
                            print(table.get_table())
                            is_found = True
                
                if not is_found:
                    break

    def find_critical_way(self, table, vector, N_processors):
            processor_vector = [0] * N_processors
            processor_story_vector = [[] for _ in range(N_processors)]
            task_vector = vector

            table = copy.copy(table)
            table._matrix = np.zeros(table.shape)
            
            for current_task in task_vector:
                current_processor = np.argmin(processor_vector)
                processor_story_vector[current_processor].append(current_task)
                processor_vector[current_processor] += current_task
                table._matrix[current_processor][len(processor_story_vector[current_processor]) - 1] = current_task
                table.get_table()
            
            #print(f"Загруженные процессоры: {processor_vector}")
            print(f"История загрузки векторов: {processor_story_vector}")
            return table
    
    def get_result(self):
        return self._result
    
    def initial_random_distribution(self, table, task_vector, type):
        table = copy.copy(table)
        table._matrix = np.zeros(table.shape)
        column_pos_vector = [0] * table.shape[0]
        for task in task_vector:
            #print(task, table.shape, table._matrix)
            current_row = np.random.randint(0, table.shape[0])
            current_column = column_pos_vector[current_row]
            column_pos_vector[current_row] += 1
            table._matrix[current_row][current_column] = task
            table.get_table()
        return table 
    
    def __get_lower_then_delta(self, vector, delta):
        for i in range(len(vector)):
            if vector[i] != 0 and vector[i] < delta:
                return (i, int(vector[i]))
        return None
    
    def get_sum_worktime_of_processes(self, table):
        return np.sum(table._matrix, axis=1)

In [56]:
table = Table()
tasks = np.random.randint(table.low_range, table.high_range, table.shape[1])
tasks

array([14, 19, 12, 13, 18, 18, 13, 11, 17, 18, 12])

### Обычный крон

In [47]:
table1 = copy.copy(table)
kron = KronMethod(table1, tasks, type='random')
kron.table.get_table()

,0,1,2,3,4,5,6,7,8,9,10,11
0,13.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,17.0,19.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,15.0,12.0,16.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,14.0,11.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
kron.calculate()
kron.table.get_table()

Дельта:  37.0
Дельта:  10.0
     0     1     2     3    4    5    6    7    8    9    10   11
0  13.0  12.0  15.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1  14.0  19.0  12.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2   0.0  12.0  16.0  19.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3  17.0  11.0  13.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
Дельта:  7.0
     0     1     2     3    4    5    6    7    8    9    10   11
0  16.0  12.0  15.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1  14.0  19.0  12.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2   0.0  12.0  13.0  19.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3  17.0  11.0  13.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
Дельта:  4.0
     0     1     2     3    4    5    6    7    8    9    10   11
0  16.0  12.0  15.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1  11.0  19.0  12.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2   0.0  12.0  13.0  19.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3  17.0  14.0  13.0   

,0,1,2,3,4,5,6,7,8,9,10,11
0,16.0,12.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12.0,19.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,11.0,13.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,17.0,14.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Крон с сортировкой по убыванию

In [49]:
table2 = copy.copy(table)
kron = KronMethod(table2, tasks, type='min')
kron.table.get_table()

До сортировки по убыванию: 


,0,1,2,3,4,5,6,7,8,9,10,11
0,14.0,11.0,17.0,12.0,13.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0
1,13.0,16.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,15.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
kron.calculate()
kron.table.get_table()

Дельта:  60.0
Дельта:  32.0
Дельта:  20.0
Дельта:  14.0
     0     1     2     3     4     5    6    7    8    9    10   11
0   0.0   0.0   0.0  15.0  13.0  12.0  0.0  0.0  0.0  0.0  0.0  0.0
1  13.0  16.0  12.0   0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
2  12.0  19.0  17.0   0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
3  19.0  14.0  11.0   0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
Дельта:  8.0
     0     1     2     3     4     5    6    7    8    9    10   11
0   0.0   0.0   0.0  19.0  13.0  12.0  0.0  0.0  0.0  0.0  0.0  0.0
1  13.0  16.0  12.0   0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
2  12.0  15.0  17.0   0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
3  19.0  14.0  11.0   0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
Дельта:  3.0
     0     1     2     3     4     5    6    7    8    9    10   11
0   0.0   0.0   0.0  19.0  12.0  12.0  0.0  0.0  0.0  0.0  0.0  0.0
1  13.0  16.0  13.0   0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
2  12.0  15.0  17.0   0.0   0.0   

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.0,0.0,0.0,19.0,12.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0
1,13.0,17.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12.0,15.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,19.0,14.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Крон после CMP

In [51]:
table3 = copy.copy(table)
kron = KronMethod(table3, tasks , CMP=True)
kron.table.get_table()


История загрузки векторов: [[15, 13, 16], [14, 12, 13], [11, 19, 19], [17, 12, 12]]


,0,1,2,3,4,5,6,7,8,9,10,11
0,15.0,13.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,14.0,12.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,11.0,19.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,17.0,12.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
kron.calculate()
kron.table.get_table()

Дельта:  10.0
     0     1     2    3    4    5    6    7    8    9    10   11
0  15.0  13.0  16.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1  19.0  12.0  13.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2  11.0  14.0  19.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3  17.0  12.0  12.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
Дельта:  3.0
     0     1     2    3    4    5    6    7    8    9    10   11
0  15.0  12.0  16.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1  19.0  12.0  13.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2  11.0  14.0  19.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3  17.0  13.0  12.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
Дельта:  2.0
     0     1     2    3    4    5    6    7    8    9    10   11
0  15.0  12.0  16.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1  19.0  12.0  12.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2  11.0  14.0  19.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3  17.0  13.0  13.0  0.0  0.0  0.0  0.0  0.0  0.0 

,0,1,2,3,4,5,6,7,8,9,10,11
0,15.0,12.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,19.0,12.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,11.0,14.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,17.0,13.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Крон после CMP с сортировкой по убыванию

In [58]:
table4 = copy.copy(table)
kron = KronMethod(table4, tasks, type='min', CMP=True)
kron.table.get_table()

История загрузки векторов: [[14, 18, 11, 12], [19, 18, 17], [12, 13, 13, 18]]
До сортировки по убыванию: 


,0,1,2,3,4,5,6,7,8,9,10
0,19.0,18.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,14.0,18.0,11.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12.0,13.0,13.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
kron.calculate()
#kron.table.get_table()
def get_table(matrix):
    table = pd.DataFrame(matrix)
    table.insert(0, 'Сумма', table.sum(axis=1))
    return table

print(get_table(kron.table._matrix))

Дельта:  0.0
   Сумма     0     1     2     3    4    5    6    7    8    9   10
0   55.0  19.0  18.0  18.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1   55.0  14.0  18.0  11.0  12.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2   55.0  12.0  13.0  13.0  17.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
